<a href="https://colab.research.google.com/github/B35783/Sergio-Mastrangelo/blob/main/C%C3%B3pia_de_Untitled3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1️⃣ Pacotes necessários



In [ ]:
install.packages(c("sidrar", "forecast", "tseries", "urca", "ggplot2", "lubridate", "tsibble", "feasts"))

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



2️⃣ Obtenção dos dados do IBGE (Produção Bens Intermediários - PBI)

In [ ]:
# Download dos dados mais recentes
dados_raw <- get_sidrar(api = "/t/1419/n1/all/v/3132/p/all/c12753/99062/d/v3132%202")

# Filtrando apenas a categoria de Bens Intermediários
pbi <- subset(dados_raw, `Categoria de uso (Código)` == "99062")

# Tratando datas e criando série temporal
pbi$Data <- ym(pbi$`Mês (Código)`)
pbi_ts <- ts(pbi$Valor, start = c(year(min(pbi$Data)), month(min(pbi$Data))), frequency = 12)
# Agora temos a série mensal de PBI do IBGE pronta.

ERROR: Error in get_sidrar(api = "/t/1419/n1/all/v/3132/p/all/c12753/99062/d/v3132%202"): could not find function "get_sidrar"


3️⃣ Separar em treinamento e teste

In [ ]:
train <- window(pbi_ts, end = c(2015,10))
test <- window(pbi_ts, start = c(2015,11))

ERROR: Error: object 'pbi_ts' not found


4️⃣ Visualização inicial

In [ ]:
autoplot(train) +
  labs(title = "Produção de Bens Intermediários - IBGE",
       y = "Índice (2012=100)", x = "") +
  theme_minimal()


ERROR: Error in autoplot(train): could not find function "autoplot"


5️⃣ Teste de estacionariedade (ADF)

In [ ]:
adf_test <- ur.df(train, type = "none", selectlags = "BIC")
summary(adf_test)
Se não for estacionária, aplicamos as diferenças:

diff_train <- diff(train)
adf_test_diff <- ur.df(diff_train, type = "none", selectlags = "BIC")
summary(adf_test_diff)



ERROR: Error in parse(text = input): <text>:3:4: unexpected symbol
2: summary(adf_test)
3: Se não
      ^


6️⃣ Teste de sazonalidade (OCSB)

In [ ]:
Vamos usar o nsdiffs():

nsdiffs(diff_train, test = "ocsb")


ERROR: Error in parse(text = input): <text>:1:7: unexpected symbol
1: Vamos usar
          ^


7️⃣ Funções de autocorrelação (ACF e PACF)

In [ ]:
ggAcf(diff_train, lag.max = 48) + theme_minimal()
ggPacf(diff_train, lag.max = 48) + theme_minimal()


ERROR: Error in ggAcf(diff_train, lag.max = 48): could not find function "ggAcf"


8️⃣ Estimação de modelos SARIMA

In [ ]:
Modelo 1 sugerido: SARIMA(2,1,2)(1,0,2)[12]

modelo1 <- Arima(train, order = c(2,1,2), seasonal = c(1,0,2))
summary(modelo1)
Modelo reduzido: SARIMA(2,1,2)(1,0,1)[12]

modelo2 <- Arima(train, order = c(2,1,2), seasonal = c(1,0,1))
summary(modelo2)
Modelo simplificado via auto.arima
modelo_auto <- auto.arima(train)
summary(modelo_auto)


ERROR: Error in parse(text = input): <text>:1:8: unexpected numeric constant
1: Modelo 1
           ^


9️⃣ Diagnóstico de resíduos

In [ ]:
checkresiduals(modelo_auto)

ERROR: Error in checkresiduals(modelo_auto): could not find function "checkresiduals"


🔟 Tratamento de outlier (quebra estrutural 2008)

In [ ]:
Criando a dummy:

dummy <- ts(ifelse(time(train) >= 2008 + 8/12 & time(train) <= 2008 + 10/12, 1, 0), start = start(train), frequency = 12)
modelo_dummy <- Arima(train, order = c(1,1,0), seasonal = c(1,0,1), xreg = dummy)
summary(modelo_dummy)


ERROR: Error in parse(text = input): <text>:1:9: unexpected symbol
1: Criando a
            ^


🔟+1 Teste Ljung-Box

In [ ]:
Box.test(residuals(modelo_dummy), lag = 11, type = "Ljung-Box")

ERROR: Error: object 'modelo_dummy' not found


🔟+2 Previsão

In [ ]:
Criar dummy para o período de teste:

dummy_test <- ts(ifelse(time(test) >= 2008 + 8/12 & time(test) <= 2008 + 10/12, 1, 0), start = start(test), frequency = 12)
Previsão:

fcast <- forecast(modelo_dummy, xreg = dummy_test, h = length(test))
autoplot(fcast) + autolayer(test, series = "Observado") + theme_minimal()
accuracy(fcast, test)


ERROR: Error in parse(text = input): <text>:1:7: unexpected symbol
1: Criar dummy
          ^
